# Assignment 'Economic Connectedness' for the course "Business Analytics Practicum I" 

Student Name:Athanasios Alexandris

Student id : p2822202

Msc : Business Analytics PT, AUEB

In this assignment we tried to replicate the requested plots from the papers listed below:

Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital I: measurement and associations with economic mobility. Nature 608, 108–121 (2022). https://doi.org/10.1038/s41586-022-04996-4.

Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital II: determinants of economic connectedness. Nature 608, 122–134 (2022). https://doi.org/10.1038/s41586-022-04997-3.

For each question we have gathered the code of our analysis and our plots, and text for describing the steps and methods.

As first moves before we begin working on the questions we imported the libraries numpy, pandas and matplotlib, then we loaded the datasets for Social Capital Atlas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#Load Data Social Capital Atlas
social_capital_county = pd.read_csv('social_capital_county.csv')
us_zip_codes= pd.read_csv('social_capital_zip.csv')
us_colleges = pd.read_csv('social_capital_college.csv')
us_high_schools = pd.read_csv('social_capital_high_school.csv')

# Question 1

First we created a copy of the initial dataset that refers to 'Economic Connectednes' by US county. Then we noticed that there are codes of that have 4 digits instead of 5, so we added a zero in the beginning of those codes, to set them to the appropriate format in order to be matched with the FIPS data that we used for our map. 

In [ ]:
us_counties=social_capital_county.copy()

In [ ]:
us_counties

In [ ]:
us_counties['county'] = us_counties['county'].astype(str)

In [ ]:
##transform observations with less than 5 digits to 5 digits addin the apropriate amount of zeros to the left (e.g 01001)
us_counties['county'] = us_counties['county'].str.zfill(5)

In [ ]:
us_counties

Next we had to calculate the decile percentiles of  that were used as ranges in the the legend of our plot, and based on them the various counties are coloured.

Then we created a new column in our dataset ('ec_cat'), that contains a string which describes the range of the 'Economic connectedness' for each county. For counties that we did not have the information of 'EC, the column 'ec_cat' is marked as 'NA'.

The whole process for the new column and its values has been made with the command 'cut' of pandas library.

In [ ]:
##define the ranges, by finding the value for each decile percentile
ec_not_na=round(us_counties.ec_county[us_counties['ec_county'].notna()],2)


percentiles=[np.percentile(ec_not_na, 10),
          np.percentile(ec_not_na, 20),
          np.percentile(ec_not_na, 30),
          np.percentile(ec_not_na, 40),
          np.percentile(ec_not_na, 50),
          np.percentile(ec_not_na, 60),
         np.percentile(ec_not_na, 70),
         np.percentile(ec_not_na, 80),
         np.percentile(ec_not_na, 90)]
percentiles

In [ ]:
##create a new column in my dataframe ('ec_cat') that contains strings that indicates the range of ec_county that the county belongs to.
##the ranges have been set based on the 10 decile percentiles of variable ec_county


us_counties["ec_cat"] = pd.cut(
    us_counties["ec_county"],
    bins=([0,np.percentile(ec_not_na, 10),
          np.percentile(ec_not_na, 20),
          np.percentile(ec_not_na, 30),
          np.percentile(ec_not_na, 40),
          np.percentile(ec_not_na, 50),
          np.percentile(ec_not_na, 60),
         np.percentile(ec_not_na, 70),
         np.percentile(ec_not_na, 80),
         np.percentile(ec_not_na, 90), us_counties["ec_county"].max()]),
    labels=["< "+ str(percentiles[0]), str(percentiles[0])+" - "+str(percentiles[1]),str(percentiles[1])+" - "+str(percentiles[2]),str(percentiles[2])+" - "+str(percentiles[3]),str(percentiles[3])+" - "+str(percentiles[4]),str(percentiles[4])+" - "+str(percentiles[5]),str(percentiles[5])+" - "+str(percentiles[6]),str(percentiles[6])+" - "+str(percentiles[7]),str(percentiles[7])+" - "+str(percentiles[8]),"> "+ str(percentiles[8])])



Next  the nan values of the column 'ec_county', were set as "NA" so this can be displayed When the pointed hovers each county. Also the 'ec_cat' value that we have already refered to, was set as catecorical column, which means that contains specific values (ec_county percentiles) that are called categories.

In [ ]:
## set all missing values of coloumn 'ec_county' as "NA"
us_counties['ec_county'] = us_counties['ec_county'].fillna("NA") 

In [ ]:
categories = ["NA","< "+ str(percentiles[0]), str(percentiles[0])+" - "+str(percentiles[1]),str(percentiles[1])+" - "+str(percentiles[2]),str(percentiles[2])+" - "+str(percentiles[3]),str(percentiles[3])+" - "+str(percentiles[4]),str(percentiles[4])+" - "+str(percentiles[5]),str(percentiles[5])+" - "+str(percentiles[6]),str(percentiles[6])+" - "+str(percentiles[7]),str(percentiles[7])+" - "+str(percentiles[8]),"> "+ str(percentiles[8])] 
us_counties['ec_cat'] = pd.Categorical(us_counties['ec_cat'], categories=categories)

us_counties['ec_cat'] = us_counties['ec_cat'].fillna("NA")

In [ ]:
us_counties["ec_cat"]

For designing our map, we had first to import an extra dataset, that we named counties, which contains information of the USA counties, as the FIPS code for the counties, their names, coordinates and more. It is a json file that was found in an example in the official site of plotly library. This json value is one of the arguements of the plotly choropleth function, that uses the FIPS codes so the counties can be recognized and when the pointed hovers each county. 

Then we used the choropleth function of plotly library. We tried to find colours that are close to the example given in the description of the question 1. 

The map created displays for each US county (scope='USA', ) the values of its 'Economic Connectednes', the name of the county and the state that it  belongs to(as hover name), and the code of the county (hover_data).  Each state is coloured based the range of economic connectedness that it bolongs to. The counties of those  we had not data are coloured with yellow, and the 'Economic Connectedness' is presented as NA. 

In [ ]:
### assign to variable 'counties' the FIPS codes for all USA counties 

import json
from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.express as px


##assign to colours variable all colour codes that we believe are similar to the example 

colours=["#FFD700", "#8B0000","#CD5C5C", "#fdbb84", "#FFE5B4", "#FFF5EE","#D4F1F4",
         "#ADD8E6", "#87CEFA", "#4169E1","#191970"]


### making the plot based on column 'ec_cat' that contains the category of the range of the county
fig = px.choropleth(us_counties, geojson=counties, locations='county', color='ec_cat',
                           scope="usa",
                           labels={'county':'county','ec_county':'economic connectedness', 'ec_cat':'Economic Connectedness'},
                            hover_data={'county':True, 'ec_county':True, 'ec_cat':False},hover_name='county_name',
                        color_discrete_sequence=colours[::-1],
                    category_orders={'ec_cat': categories[::-1]}
                   )




fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# QUESTION 2


For question 2 we first made a copy ('us_counties_q2') of the initial usocial_capital_county dataset, as in the previous copy we had made changes that would affect our work in this qustions (e.g filling nan values as NA in ec_county).

We also had to load also the dataset 'county_outcomes_simple.csv' from Opportunity Atlas as a new dataframe 'county_outcomes_simple', that contains the data regarding the upward income mobility of the us counties.

In [ ]:
###we make a copy of the initial dataset with us_counties
us_counties_q2=social_capital_county.copy()

As in the previous question for the social_capital_county dataset, we transform the 4 digits values of column 'county' to 5 digits, adding a zero in the beginning of the string. 
The second dataset had the columns 'State' and 'County', that had to be concatenated to create the FIPS county code so we can match them with the respective column of the other dataset. The column 'state' should contain two digit stings, and the county three digit strings. So we implemented the same method, adding a zero in the one digit string of states, and one or two zeros in the 1 or two digits sting of counties respectively, and then from the concatenation of those columns the 'FIPS_county' column was created, that involves same format codes with the first dataset, and then we were able to merge them.

In [ ]:
us_counties_q2['county'] = us_counties_q2['county'].astype(str)

In [ ]:
us_counties_q2['county'] = us_counties_q2['county'].str.zfill(5)

In [ ]:
county_outcomes_simple = pd.read_csv('county_outcomes_simple.csv')


In [ ]:
county_outcomes_simple.describe()

In [ ]:
county_outcomes_simple['state'] = county_outcomes_simple['state'].astype(str)
county_outcomes_simple['county'] = county_outcomes_simple['county'].astype(str)


In [ ]:
county_outcomes_simple['state'] = county_outcomes_simple['state'].str.zfill(2)
county_outcomes_simple['county'] = county_outcomes_simple['county'].str.zfill(3)

In [ ]:
###we concatenate the two columns 'state' and county

county_outcomes_simple['FIPS_county']=county_outcomes_simple['state'].astype(str).str.cat(county_outcomes_simple['county'], sep='')

In [ ]:
##we moved the new column 'FIPS_county' in the third place of the dataframe
new_col = county_outcomes_simple.pop('FIPS_county')
county_outcomes_simple.insert(2,'FIPS_county', new_col)



Then a new dataframe named joined_df was created, by joining the two datasets we have spoke about. The merge was done on columns  'FIPS_county' for the 'county_outcomes_simple' dataset and 'county' for the us_counties_q2 dataset. The columns that we decided to keep are the following : 'FIPS_county'(code of county), 'czname' (community zone name), 'kfr_pooled_pooled_p25' (upward economic mobility), 'ec_county' (economic connectedness), 'pop2018' (population of the county for 2018).

In [ ]:
##We created a new dataframe that is joined by the two existing dataframe on columns 'FIPS_county' and 'county' respectively, which contain the same information and in the same format

joined_df = pd.merge(county_outcomes_simple[['FIPS_county','czname', 'kfr_pooled_pooled_p25']], us_counties_q2[['ec_county', 'pop2018','county']], left_on='FIPS_county',right_on='county')
del joined_df['county']
joined_df


After dropping the rows of our dataset that contained nan values, we sort our dataset on the pop2018 column and then we kept the 200 first rows (head function), as the question required the data of the 200 most populous us counties.


In [ ]:
##Drop rows that contain nan values.
joined_df=joined_df.dropna()

In [ ]:
joined_df.describe()

In [ ]:
##We sorteded the dataframe in descending order and we kept only the the first 200 observations (counties)

joined_df_200=joined_df.sort_values(by='pop2018',ascending=False)
joined_df_200=joined_df_200.head(200)
joined_df_200

Then we created a sub dataframe that contained only the rows, that refer to the 5 states thar were showing in the example. Then we tried to identify the specific county for each state thar are annotated in the example by observing example's map. According to their coordinates (EC, kfr), the specific FIPS codes of the states, 'San Francisco','Indianapolis','Minneapolis','New York','Salt Lake City, are 06075, 18097, 27053, 36061, 49035 respectively.

In [ ]:
##We created a dataframe containing only the observatins that have 'czname' one of them that are presented in example plot, 
##and then we tried to identify from their coordinates ('ec_county','kfr_pooled_pooled_p25'), the specific observations that are presented in the example.

states_sample=['San Francisco','Indianapolis','Minneapolis','New York','Salt Lake City']
states_sample_data = joined_df_200[joined_df_200["czname"].isin(states_sample)].sort_values(by='czname',ascending=False)
states_sample_data


Finally, to create our plot, we used the lmplot function of seaborn library, which designs also the regression line along with the points of the scaterplot. After creating the lpmplot with the regression line, we also renamed the x,y axis to be the same to the paper's plot, and also we annotated as better as we could the counties pointed in the example.

In [ ]:
import seaborn as sns

g= sns.lmplot(x='ec_county', y='kfr_pooled_pooled_p25', data=joined_df_200,aspect=2)

# Add arrows pointing to the highlighted observations
plt.annotate(str(states_sample[0]), xy=(joined_df_200.ec_county[joined_df_200.FIPS_county=='06075'], joined_df_200.kfr_pooled_pooled_p25[joined_df_200.FIPS_county=='06075']), xytext=(1.27, 0.403888), arrowprops=dict(arrowstyle="-|>", color="black"))
plt.annotate(str(states_sample[1]), xy=(joined_df_200.ec_county[joined_df_200.FIPS_county=='18097'], joined_df_200.kfr_pooled_pooled_p25[joined_df_200.FIPS_county=='18097']), xytext=(0.699, 0.332), arrowprops=dict(arrowstyle="-|>", color="black"))
plt.annotate(str(states_sample[2]), xy=(joined_df_200.ec_county[joined_df_200.FIPS_county=='27053'], joined_df_200.kfr_pooled_pooled_p25[joined_df_200.FIPS_county=='27053']), xytext=(1.012, 0.367), arrowprops=dict(arrowstyle="-|>", color="black"))
plt.annotate(str(states_sample[3]), xy=(joined_df_200.ec_county[joined_df_200.FIPS_county=='36061'], joined_df_200.kfr_pooled_pooled_p25[joined_df_200.FIPS_county=='36061']), xytext=(0.792, 0.5), arrowprops=dict(arrowstyle="-|>", color="black"))
plt.annotate(str(states_sample[4]), xy=(joined_df_200.ec_county[joined_df_200.FIPS_county=='49035'], joined_df_200.kfr_pooled_pooled_p25[joined_df_200.FIPS_county=='49035']), xytext=(0.91105, 0.509), arrowprops=dict(arrowstyle="-|>", color="black"))

g.set_xlabels("Economic Connectedness")
g.set_ylabels("Predicted Household Income Rank\nFor Children with Parents at 25th Income Percentile ")

sns.set_style("darkgrid")

plt.show()




# Question 3

For this question we had to compile data from the replication package of the papers with data from the Social Capital Atlas Datasets. We loaded the 'zip_covariates.dta' as a data frame named zip_covariates. From this data frame we need the data regarding the median household income in ZIP code and the upward mobility. 
The second dataset that we used is the us_zip_codes dataframe that we created in the beginning of the assignment. From this data frame we need the data regarding the economic connectedness in ZIP code.

In [ ]:
zip_covariates = pd.read_stata('zip_covariates.dta')

In [ ]:
zip_covariates

In [ ]:
us_zip_codes

Next we merged the two datasets on 'zip' column that contained the ZIP codes. The columns that we decided to keep are the following: zip (ZIP codes), ec_zip (Economic Connectedness) pop2018_x(population per zip code in 2018 of the first dataset)	kfr_pooled_pooled_p25(upward income mobility per zip), med_inc_2018(median income per zip), pop2018_y (population per zip code in 2018 of the second dataset). As both datasets contained the pop2018 column, we decided to keep them both in the new dataframe that we created, and since we checked that there are no any difference between these two columns we kept just one of them (pop2018_x).

In [ ]:
zip_merged=pd.merge(us_zip_codes[['zip','ec_zip','pop2018']], zip_covariates[['zip','kfr_pooled_pooled_p25','med_inc_2018','pop2018']], on='zip')

In [ ]:
zip_merged

In [ ]:
(~(zip_merged['pop2018_x'] - zip_merged['pop2018_y']).isin([0])).sum()

In [ ]:
del zip_merged['pop2018_y']

Next we dropped the rows that contained nan values. The column that described the percentage of upward income mobility in zip code was multiplied by 100. Then we calculated the 4 quantiles (20%, 40%, 60%, 80%), of the column that contained the data of upward mobility. We created a new column in our data frame, through pd.cut function, which contains the quantile ranges of upward mobility, that every zip belong to based on their upward income mobility values, as strings ("<38", "38-41", "41-44","44-48", ">48"). The points of our scaterplot will be coloured based on this column. More particularly, for every range there is one colour.

In [ ]:
zip_merged.dropna()

In [ ]:
zip_merged=zip_merged.dropna()

In [ ]:
zip_merged['kfr_pooled_pooled_p25']=100*zip_merged['kfr_pooled_pooled_p25']

In [ ]:
kfr_quantiles=[round(np.percentile(zip_merged.kfr_pooled_pooled_p25, 20)),round(np.percentile(zip_merged.kfr_pooled_pooled_p25, 40)),round(np.percentile(zip_merged.kfr_pooled_pooled_p25, 60)),round(np.percentile(zip_merged.kfr_pooled_pooled_p25, 80))]

In [ ]:
kfr_quantiles

In [ ]:
categories_sample=zip_merged.kfr_pooled_pooled_p25

zip_merged["kfr_cat"] = pd.cut(
    zip_merged["kfr_pooled_pooled_p25"],
    bins=([0,np.percentile(categories_sample, 20),
          np.percentile(categories_sample, 40),
          np.percentile(categories_sample, 60),
          np.percentile(categories_sample, 80),
         categories_sample.max()]),
    labels=["< " + str(kfr_quantiles[0]) ,str(kfr_quantiles[0]) + " - " + str(kfr_quantiles[1]), str(kfr_quantiles[1]) + " - " + str(kfr_quantiles[2]), str(kfr_quantiles[2]) + " - " + str(kfr_quantiles[3]),"> " + str(kfr_quantiles[3])])

Next we had to filtered our data like the authors of the papaer did, keeping the rows of the dataframe whose median income is between the 5th and the 95th percentile. More over we dropped also the rows whose population was under the 10th percentile. 

In [ ]:
filtered_zip_merged=zip_merged[(zip_merged['med_inc_2018']>=np.percentile(zip_merged.med_inc_2018, 5)) & (zip_merged['med_inc_2018']<=np.percentile(zip_merged.med_inc_2018, 95)) & (zip_merged['pop2018_x']>=np.percentile(zip_merged.pop2018_x, 10))]

In [ ]:
filtered_zip_merged

In [ ]:
filtered_zip_merged.med_inc_2018.describe()

Finally, we created our scaterplot through seaborn library. In x-axis it is shown median household income in Zip code and in y-axis it is shown the econommic connectedness of the zip. Each point of the scatterplot is coloured based on the range of the upward mobility that it belongs to. The colours used are as close to the example's colours as possible. We also adjusted the inc level of the points through alpha option, we renamed the y-axis and the x-axis, and we adjusted the figure size of the plot.

In [ ]:
labels2=["< " + str(kfr_quantiles[0]) ,str(kfr_quantiles[0]) + " - " + str(kfr_quantiles[1]), str(kfr_quantiles[1]) + " - " + str(kfr_quantiles[2]), str(kfr_quantiles[2]) + " - " + str(kfr_quantiles[3]),"> " + str(kfr_quantiles[3])]

h=sns.scatterplot(data=filtered_zip_merged, x='med_inc_2018', y='ec_zip',hue='kfr_cat',palette=["#2E2E7A", "#0077BE", "#FFE4C4", "#FF8C00","#DC143C"], hue_order=labels2[::-1],alpha=0.7)

plt.legend(loc='lower right', title="Upward Mobility")

plt.xlabel("Median Household Income in ZIP Code (US$)")
plt.ylabel("Economic Connectedness")

sns.set_style("darkgrid")
sns.set(rc={'figure.figsize':(11,7.5)}) 




plt.show()



# Question 4



For this question we had to use the dataset of 'social_capital_high_school.csv' that we named us_high_schools.
As it is explained in the description of question 4, we had to calculate the share of students with high parental Socioeconomic Status (SES), against the friending bias of students of low parental SES. 



In [ ]:
us_high_schools

As described for the calculation of the share, we divided the column 'exposure_parent_ses_hs' (mean exposure to high-parental-SES individuals by high school) by 2 and multiply it to 100.
The  friending bias of students of low parental SES, is represented by column 'bias_parent_ses_hs'. This column was also multiplied to 100.
We also dropped the rows tha contain nan values for the columns that we were interested in (exposure_parent_ses_hs, bias_parent_ses_hs) and the we proceeded with the implementation of our plot.

In [ ]:
us_high_schools['share']=100*us_high_schools['exposure_parent_ses_hs']/2
data_highschool=us_high_schools[us_high_schools['exposure_parent_ses_hs'].notna()]
data_highschool=us_high_schools[us_high_schools['bias_parent_ses_hs'].notna()]
data_highschool['bias_parent_ses_hs']=100*data_highschool['bias_parent_ses_hs']


In [ ]:
us_high_schools

For our scatterplot we used the ggplot, of the plotnine library. We also used the adjustText libary, in order to use adjust_text_dict that helps for avoiding the overlap between the labels of the annotated points (high schools) that are referred in the description of the question. Through adjust_text_dict we also designed the arrows tha point the specific high schools.
We renamed x,y axis to be the same to paper's plot, we adjust the inc level of the points through alpha option, we used the geom_point to point the high schools that should be pointd (those with cyan colour).Also we used 'geom_label' option to adjust the labels of the annotated points, and in geom_label we also used the adjust_text_dict that we mentioned above. Finally we adjusted the figursize and x-axis to be similar with example's plot. 


In [ ]:
# Create the scatterplot with reversed y-axis and annotations
import adjustText
from plotnine import *

adjust_text_dict = {
    'expand_points': (-2, -2.3),
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'cyan',
        'lw': 1,
        'alpha': 0.5
    },   
}

annotated_data = data_highschool[data_highschool['high_school'].isin(['00941729', '060474000432', '170993000942', '170993001185', '170993003989', '171449001804', '250327000436', '360009101928', '370297001285', '483702004138', '250843001336', '062271003230', '010237000962', '00846981', '00852124'])]

(ggplot(data_highschool, aes(x='share', y='bias_parent_ses_hs'))
 + geom_point(alpha=0.1)
 + scale_y_reverse()
 +geom_label(mapping=aes(label='high_school_name') ,data=annotated_data,
             adjust_text=adjust_text_dict,
               alpha=1,size=8)
+ labs(x='Share of high-parental-SES students (%)', y='Friending bias among low-parental-SES students(%)')
+ geom_point(data=annotated_data, alpha=0.5, color='cyan',size=2)
+ theme(figure_size=(11, 5))
+ scale_x_continuous(limits=[0, 100], breaks=[0, 20, 40, 60, 80]
                           )


)

# Question 5

In this question we had to present binned scatter plots of friending bias against racial diversity within colleges and neighbourhoods (ZIP codes). The two dataframes that we used are the 'zip_merged_q5' and 'college_merged', that we will explain how they are created in the next paragraphs. Our goal is to create a new (concatenated) data frame from the two datasets that we discussed above with columns 'HHI' (Racial diversity based on the Herfindahl-Hirschman Index), 'bias', 'weight', and a column named 'group' that defines whether the data come from the the college dataset or the zip dataset.

For creating the 'zip_merged_q5' dataframe, we merged the dataframes 'us_zip_codes', 'zip_covariates', that we used on previous questions, on the column zip.




In [ ]:
zip_merged_q5=pd.merge(us_zip_codes, zip_covariates, on='zip')

In [ ]:
zip_merged_q5

Next we had to create a new column in this dataframe that refers to the racial diversity which comes from the formula explained in the question's description.

In [ ]:
zip_merged_q5['HHI']=1-(zip_merged_q5['share_white_2018']**2+zip_merged_q5['share_black_2018']**2+zip_merged_q5['share_natam_2018']**2+zip_merged_q5['share_asian_2018']**2+zip_merged_q5['share_hawaii_2018']**2+zip_merged_q5['share_hispanic_2018']**2)

In [ ]:
zip_merged_q5

As the merged data frame has two columns that they contain the same type of information num_below_p50_x, num_below_p50_y) in which we are interested in , we had to check if they differ. So we created a new column 'dif_num_below_p50' that refers to their difference. Function describe shows that the min and max difference are both very low numbers, and given that those columns refer to number of people these differences are insignificant. So we end up that both columns are the same and we can keep just the one of them (e.g num_below_p50_x).

In [ ]:
zip_merged_q5['dif_num_below_p50']=zip_merged_q5['num_below_p50_x'] - zip_merged_q5['num_below_p50_y']

In [ ]:
zip_merged_q5['dif_num_below_p50'].describe()

Next we keep from the dataframe only the columns that we are interested in and we talked about previously. Then we renamed these columns acordingly. 
We also created a new column 'group' that is filled with the string "Neighborhood", so in the next steps when we were planning to concatenate the two dataframes that we referred to in the beginning, we can seperate the rows that refer to zips from rows that refer to colleges.

In [ ]:
zip_merged_q5=zip_merged_q5[['HHI','num_below_p50_x','nbhd_bias_zip']]

In [ ]:
zip_merged_q5 = zip_merged_q5.rename(columns={'num_below_p50_x': 'weight','nbhd_bias_zip':'bias'})

In [ ]:
zip_merged_q5['group']="Neighborhood"

In [ ]:
zip_merged_q5

Next we loaded the 'college_characteristics.dta' from the replication package, and we created a data frame named 'college_characteristics'.
Then we merged this to the 'us_colleges' dataframe on  column 'college'. 

In [ ]:
college_characteristics = pd.read_stata('college_characteristics.dta')

In [ ]:
college_characteristics

In [ ]:
college_merged=pd.merge(college_characteristics,us_colleges,on='college')

Once again we had to create a new column 'HHI' that describes the racial diversity, and it is calculated with similar way as the previous. The main difference was that we had just to define first the share of white people 'white_share_fall_2000' that it is calculated as one minus the sum of shares of the other categories.

In [ ]:
college_merged['white_share_fall_2000']=1-(college_merged['black_share_fall_2000']+college_merged['hisp_share_fall_2000']+college_merged['asian_or_pacific_share_fall_2000'])

In [ ]:
college_merged['HHI']=1-(college_merged['black_share_fall_2000']**2+college_merged['hisp_share_fall_2000']**2+college_merged['asian_or_pacific_share_fall_2000']**2+college_merged['white_share_fall_2000']**2)

As we did in the previous data frame we keep just the columns that we are interested in, and also we added the a new column 'group' filled with the string 'college'. The remained columns of the dataset have been renamed accordingly to the first dataset, so we can concatenate them.

In [ ]:
college_merged=college_merged[['HHI','mean_students_per_cohort_x','bias_own_ses_college',]]

In [ ]:
college_merged

In [ ]:
college_merged['group']="College"

In [ ]:
###rename columns

college_merged = college_merged.rename(columns={'mean_students_per_cohort_x':'weight','bias_own_ses_college': 'bias'})

We created a new dataframe named 'merged_df_q5', that is the concatenation of the data frames 'zip_merged_q5', 'college_merged'. For the 'merged_df_q5' we kept only the rows that they had not nan values.

In [ ]:
merged_df_q5 = pd.concat([zip_merged_q5, college_merged])

In [ ]:
merged_df_q5

In [ ]:
merged_df_q5=merged_df_q5.dropna()

We multiplied the column 'bias' by 100 to be the same as the example

In [ ]:
merged_df_q5['bias']=merged_df_q5['bias']*100

In [ ]:
merged_df_q5

We calculated the mean of the weights in order to use this in our lmplot that is followed next.

In [ ]:
mean_HHI = np.average( merged_df_q5['HHI'], weights=merged_df_q5['weight'])

In [ ]:
mean_HHI

Finally, through seaborn library we created our lmplolt. As x we set the values we put the 'HHI' column and the values are binned in 20 5%-percentiles, as y= we set the column y we put the 'bias' column and as hue the group in order to create two different regression lines. We renamed the x,y labels to be the same with the example, we adjusted the x-axis to the same values with the example, and finally we adjusted the legend to the upper left side of the plot.

In [ ]:
f=sns.lmplot(x="HHI", y="bias", hue="group", data=merged_df_q5,
           markers=["o","D"],x_bins=20, palette=['#FF8C00', '#4682B4'], ci=None,  scatter_kws={"s":100*mean_HHI})

f.set_xlabels("Racial Diversity (Herfindahl-Hirschman Index) in Group")
f.set_ylabels("Friending Bias among Low-SES Individuals (%)")
f.set(xlim=(0, 0.75))
f.ax.legend(loc='upper left')



plt.show()